In [1]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg, F
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx
import random
import csv

sys.path.append('/home/galm/software/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457
q = Query.objects.get(pk=qid)
docs = Doc.objects.filter(query=q)
techs = list(Technology.objects.filter(
    pk__lt=11
).values_list('pk',flat=True))

In [2]:
tdocs = Doc.objects.filter(
    technology__in=techs
) | Doc.objects.filter(
    query__technology__in=techs
)

tidocs = tdocs.filter(innovation__isnull=False) | tdocs.filter(query__innovation__isnull=False)

In [3]:
tdocs.distinct().count()


7334

In [4]:
tidocs.distinct().count()

3125

In [5]:
ins = Innovation.objects.all()
for i in ins:
    docs = tdocs.filter(innovation=i) | tdocs.filter(query__innovation=i)
    radocs = docs.filter(docownership__query__innovation=i,docownership__relevant__gt=4)
    rdocs = docs.filter(docownership__query__innovation=i,docownership__relevant=5)
    print('\n########\n')
    print(i.name)
    d = docs.distinct().count()
    rd = rdocs.distinct().count()
    ra = radocs.distinct().count()
    print("{} docs from query, of which {} were rated and {} relevant".format(d, ra, rd))


########

RD
1564 docs from query, of which 1562 were rated and 1533 relevant

########

Demos
259 docs from query, of which 257 were rated and 53 relevant

########

Scaleup
759 docs from query, of which 755 were rated and 339 relevant

########

Demandpull
424 docs from query, of which 423 were rated and 167 relevant

########

Nichemarkets
55 docs from query, of which 54 were rated and 18 relevant

########

Publicaccept
1310 docs from query, of which 1023 were rated and 153 relevant

########

Innovationgeneral
3122 docs from query, of which 0 were rated and 0 relevant

########

RDNETS
0 docs from query, of which 0 were rated and 0 relevant

########

DemosNETS
0 docs from query, of which 0 were rated and 0 relevant

########

ScaleupNETS
1 docs from query, of which 0 were rated and 0 relevant

########

DemandpullNETS
0 docs from query, of which 0 were rated and 0 relevant

########

NichemarketsNETS
2 docs from query, of which 2 were rated and 1 relevant

########

Publicaccept

In [6]:
ins = Innovation.objects.exclude(name__icontains="NETS")



with open("NETS_INNOVATION.csv","w") as wf:
    f = csv.writer(wf)
    f.writerow(['technology','innovation','total_docs','reviewed','relevant'])

    ins = Innovation.objects.all()
    for t in techs:
        techdocs = Doc.objects.filter(technology=t) | Doc.objects.filter(query__technology=t)
        tadocs = Doc.objects.filter(docownership__query__technology=t,docownership__relevant__gt=0)
        trdocs = Doc.objects.filter(
            docownership__query__technology=t,
            docownership__relevant=1
        ) | Doc.objects.filter(
            technology=t
        )
        print('\n########\n')
        tname = Technology.objects.get(pk=t).name
        print(tname)
        d = techdocs.distinct('UT').count()
        rd = trdocs.distinct('UT').count()
        ra = tadocs.distinct('UT').count()
        print("{} docs from queries, of which {} were rated and {} relevant".format(d, ra, rd))
        
        f.writerow([tname,'total',d,ra,rd])

        for i in ins:
            ## all techs
            
            docs = tdocs.filter(innovation=i) | tdocs.filter(query__innovation=i)
            radocs = docs.filter(docownership__query__innovation=i,docownership__relevant__gt=4)
            rdocs = docs.filter(docownership__query__innovation=i,docownership__relevant=5)
            d = docs.distinct().count()
            rd = rdocs.distinct().count()
            ra = radocs.distinct().count()
            
            f.writerow(['total',i.name,d,ra,rd])
            
            
            #######
            
            ## this tech
            docs = techdocs.filter(innovation=i) | techdocs.filter(query__innovation=i)
            rdocs = docs.filter(docownership__query__innovation=i,docownership__relevant=5)
            print('\n\t########\n')
            print('\t'+i.name)
            d = docs.distinct().count()
            rd = rdocs.distinct().count()
            print("\t{} docs from queries, of which {} relevant".format(d, rd))
            
            f.writerow([tname,i.name,d,ra,rd])
            
            


########

Synonyms
506 docs from queries, of which 422 were rated and 240 relevant

	########

	RD
	36 docs from queries, of which 35 relevant

	########

	Demos
	9 docs from queries, of which 4 relevant

	########

	Scaleup
	38 docs from queries, of which 26 relevant

	########

	Demandpull
	36 docs from queries, of which 17 relevant

	########

	Nichemarkets
	4 docs from queries, of which 3 relevant

	########

	Publicaccept
	36 docs from queries, of which 9 relevant

	########

	Innovationgeneral
	84 docs from queries, of which 0 relevant

	########

	RDNETS
	0 docs from queries, of which 0 relevant

	########

	DemosNETS
	0 docs from queries, of which 0 relevant

	########

	ScaleupNETS
	0 docs from queries, of which 0 relevant

	########

	DemandpullNETS
	0 docs from queries, of which 0 relevant

	########

	NichemarketsNETS
	0 docs from queries, of which 0 relevant

	########

	PublicacceptNETS
	0 docs from queries, of which 0 relevant

	########

	InnovationgeneralNETS
	0 docs 